# Using critical sections
A [critical section](https://en.wikipedia.org/wiki/Linearizability) is a region of code that should not run in parallel. For example, the increment of a variable is not considered an [atomic operation](https://en.wikipedia.org/wiki/Linearizability), so, it should be performed using [mutual exclusion](https://en.wikipedia.org/wiki/Mutual_exclusion).

## What happens when mutial exclusion is not used in critical sections?

### Using threads
All Python’s built-in data structures (such as lists, dictionaries, etc.) are thread-safe. However, other user's data structures implemented in Python, or simpler types like integers and floats, cannot be accesed concurrently.

In [1]:
# Two threads that have a critical section executed in parallel without mutual exclusion.
# This code does not work!

import threading
import time

counter = 10

def task_1():
    global counter
    for i in range(1000000):
        counter += 1
        
def task_2():
    global counter
    for i in range(1000001):
        counter -= 1
        
thread_1 = threading.Thread(target=task_1)
thread_2 = threading.Thread(target=task_2)

thread_1.start()
thread_2.start()
print("(Both threads started)")

thread_1.join()
thread_2.join()
print("\nBoth threads finished")

print('counter =', counter)

(Both threads started)

Both threads finished
counter = -99559


The same example, using mutual exclusion (using a [lock](https://docs.python.org/3/library/threading.html#lock-objects)):

In [5]:
# Two threads that have a critical section executed sequentially.

import threading
import time

lock = threading.Lock()
counter = 10

def task_1():
    global counter
    for i in range(1000000):
        with lock:
            counter += 1
        
def task_2():
    global counter
    for i in range(1000001):
        with lock:
            counter -= 1
        
thread_1 = threading.Thread(target=task_1)
thread_2 = threading.Thread(target=task_2)

now = time.perf_counter()  # Real time (not only user time)
thread_1.start()
thread_2.start()
print("(Both threads started)")

thread_1.join()
thread_2.join()
print("\nBoth threads finished")
elapsed = time.perf_counter() - now
print(f"\nelapsed {elapsed:0.2f} seconds.")

print('counter =', counter)

(Both threads started)

Both threads finished

elapsed 6.91 seconds.
counter = 9


### Using processes

In [3]:
# Two processes that have a critical section executed sequentially

import multiprocessing
import time
import ctypes

def task_1(lock, counter):
    for i in range(10000):
        lock.acquire()
        try:
            counter.value += 1
        finally:
            lock.release()
        
def task_2(lock, counter):
    for i in range(10001):
        lock.acquire()
        try:
            counter.value -= 1
        finally:
            lock.release()

lock = multiprocessing.Lock()

manager = multiprocessing.Manager()
counter = manager.Value(ctypes.c_int, 10)

process_1 = multiprocessing.Process(target=task_1, args=(lock, counter))
process_2 = multiprocessing.Process(target=task_2, args=(lock, counter))

process_1.start()
process_2.start()
print("(Both tasks started)")

process_1.join()
process_2.join()
print("\nBoth tasks finished")

print('counter =', counter.value)

(Both tasks started)

Both tasks finished
counter = 9


### Using coroutines
Notice that the lock is not needed at all.

In [4]:
import asyncio
import time

counter = 10

async def task_1():
    global counter
    for i in range(1000000):
        counter += 1
        
async def task_2():
    global counter
    for i in range(1000001):
        counter -= 1

t1 = task_1()
t2 = task_2()

now = time.perf_counter()  # Real time (not only user time)
await asyncio.gather(task_1(), task_2())
elapsed = time.perf_counter() - now
print(f"\nelapsed {elapsed:0.2f} seconds.")
print('counter =', counter)


elapsed 0.26 seconds.
counter = 9


## An example with classes

In [ ]:
import logging
import random
import threading
import time

class Counter:
    
    def __init__(self, start=0):
        self.lock = threading.Lock()
        self.value = start

    def increment(self):
        logging.debug('Waiting for lock')
        self.lock.acquire()
        try:
            logging.debug('Acquired lock')
            self.value = self.value + 1
        finally:
            self.lock.release()

def worker(c):
    for i in range(2):
        pause = random.random()
        logging.debug('Sleeping %0.02f', pause)
        time.sleep(pause)
        c.increment()
    logging.debug('Done')


logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

counter = Counter()
for i in range(2):
    t = threading.Thread(target=worker, args=(counter,))
    t.start()

logging.debug('Waiting for worker threads')
main_thread = threading.main_thread()
for t in threading.enumerate():
    if t is not main_thread:
        t.join()
logging.debug('Counter: %d', counter.value)